# Effect of calibration equation on kinetic parameters

In this analysis the influence of calibration model selection on the resulting parameters should was investigated. Each parameter estimation scenario in this work relies on the conversion from absorption data into concentration data. Therefore, a standard of the analyte, which was used to measure the enzyme activty, was provided with each kinetic experiment. Usually, a linear relationship between the absobance signal and concentration is assumed by experimentalists. Therefore, a linear calibration is applied. 

Experimentalists often restrict themselfs to only use the so called 'linear' range of the photometer below an absorption of 1. Thus, sacrificing a significant ammount of the measurement range of the analytical device. 

## Data
- chymotrypsin
- SLAC
- trametes
- glucosidase
- 



## Calibration

## Inactivation

### Effekt of calibration temperature and pH on kinetic parameters

In [ ]:
enzmldoc = enzmldoc_absorption_dict["3.0 45.0"]

standards_variations = list(filter(lambda x: x.calibration_data.pH == 3.0, standard_list))
[standards_variations.append(x) for x in list(filter(lambda x: x.calibration_data.temperature == 45 and x.calibration_data.pH < 6, standard_list))];


In [ ]:

kinetics = []
for standard in standards_variations:
    print(copy.__file__)
    doc = copy.deepcopy(enzmldoc)
    standard.apply_to_EnzymeML(doc, species_id="s0", ommit_nan_measurements=True)
    kinetic = (ParameterEstimator.from_EnzymeML(doc, reactant_id="s0", measured_species="substrate"))
    kinetic.fit_models(enzyme_inactivation=True, only_irrev_MM=True)
    kinetics.append(kinetic)




## Using linear calibration instead of 3rd polynominal model


In [ ]:
models = [None, "Linear"]
kinetics=[]
for model in models:
    print(copy.__file__)
    doc = copy.deepcopy(enzmldoc)
    standards_variations[4].apply_to_EnzymeML(doc, species_id="s0", ommit_nan_measurements=True, model_name=model)
    kinetic = (ParameterEstimator.from_EnzymeML(doc, reactant_id="s0", measured_species="substrate"))
    kinetic.fit_models(enzyme_inactivation=True, only_irrev_MM=True)
    kinetics.append(kinetic)

# Get kinetic parameters of all datasets
kcat = []
kcat_std = []
Km = []
Km_std = []
ki = []
ki_std = []
temperature = [45,45]
pH = [3,3]
for result in kinetics:
    params = result.get_parameter_dict()

    kcat.append(params["k_cat"].value)
    kcat_std.append(params["k_cat"].stderr)

    Km.append(params["Km"].value)
    Km_std.append(params["Km"].stderr)

    ki.append(params["K_ie"].value)
    ki_std.append(params["K_ie"].stderr)


df = pd.DataFrame.from_dict({
    'pH':pH, 
    "temperature":temperature, 
    'kcat [1/s]':kcat, 
    'kcat stderr':kcat_std, 
    'Km [uM]':Km, 
    'Km stderr':Km_std, 
    "Enzyme inactivation [1/s]":ki,
    "Enzyme inactivation std":ki_std})

df["kcat/Km"] = df["kcat [1/s]"] / df["Km [uM]"]
kcat_km_stderr =((df["kcat stderr"]/df["kcat [1/s]"])**2+(df["Km stderr"]/df["Km [uM]"])**2)**0.5 * df["kcat/Km"]
df["kcat/Km stderr"] = kcat_km_stderr

ph_map = sns.color_palette("husl")
ph_colors = [ph_map[(x)] for x in np.arange(6)]
pH_dict = dict(zip([3.0, 3.5, 4.0, 4.5, 5.0, 5.5], ph_colors))
pH_tuples = [pH_dict[x] for x in df["pH"].values]

temperature_map = matplotlib.cm.get_cmap('coolwarm')
colors = [temperature_map(x) for x in np.linspace(0,1,5)]
color_dict = dict(zip([25, 30, 35, 40 ,45], colors))
color_tuples = [color_dict[x] for x in df["temperature"]]

ax = plt.gca()
for i, (km, kcat, km_error, kcat_error, color, label) in enumerate(zip(df["Km [uM]"], df["kcat [1/s]"], df["Km stderr"], df["kcat stderr"], pH_tuples, df["pH"].values)):

    ax.scatter(km, kcat, color = color, label = label if not i % 5 else "")
    ax.errorbar(km, kcat, yerr=kcat_error,xerr=km_error, fmt=".", ecolor=color, markersize=0)
plt.legend(title = "pH")
plt.ylabel("kcat [1/s]")
plt.xlabel("Km [uM]")
plt.xlim([0,200])

In [ ]:
subset_enzymeml = [copy.deepcopy(x) for x in enzmldoc_absorption_dict.values() if x.getReaction("r0").temperature == 45]
models = [None, "Linear"]
calibration_data = [x for x in standard_curve_dict.values() if x.calibration_data.temperature == 45 and x.calibration_data.pH < 6]
applied_model = {"best model": None, "Linear": "Linear"}

kinetics = []
model_names = []
for doc, standard in zip(subset_enzymeml, calibration_data):
    for name, value in applied_model.items():
        d = copy.deepcopy(doc)
        standard.apply_to_EnzymeML(d, species_id="s0", ommit_nan_measurements=True, model_name=value)
        kinetic = (ParameterEstimator.from_EnzymeML(d, reactant_id="s0", measured_species="substrate"))
        kinetic.fit_models(enzyme_inactivation=True, only_irrev_MM=True);
        kinetics.append(kinetic)
        kinetic.visualize()
        model_names.append(name)



model_names


In [ ]:
kcat = []
kcat_std = []
Km = []
Km_std = []
ki = []
ki_std = []
temp = []
calibration_model = np.tile(np.array(["best model", "linear calibration"]), 6)
pH = np.repeat(np.array([3,3.5,4,4.5,5,5.5]), 2)
for result in kinetics:
    params = result.get_parameter_dict()
    temp.append(result.data.temperature)

    kcat.append(params["k_cat"].value)
    kcat_std.append(params["k_cat"].stderr)

    Km.append(params["Km"].value)
    Km_std.append(params["Km"].stderr)

    ki.append(params["K_ie"].value)
    ki_std.append(params["K_ie"].stderr)


df = pd.DataFrame.from_dict({
    'pH':pH, 
    "calibration model":calibration_model, 
    'kcat [1/s]':kcat, 
    'kcat stderr':kcat_std, 
    'Km [uM]':Km, 
    'Km stderr':Km_std, 
    "Enzyme inactivation [1/s]":ki,
    "temperature": temp,
    "Enzyme inactivation std":ki_std})

df["kcat/Km"] = df["kcat [1/s]"] / df["Km [uM]"]
kcat_km_stderr =((df["kcat stderr"]/df["kcat [1/s]"])**2+(df["Km stderr"]/df["Km [uM]"])**2)**0.5 * df["kcat/Km"]
df["kcat/Km stderr"] = kcat_km_stderr

df
        

In [ ]:
ph_map = sns.color_palette("husl")
ph_colors = [ph_map[(x)] for x in np.arange(6)]
pH_dict = dict(zip([3.0, 3.5, 4.0, 4.5, 5.0, 5.5], ph_colors))
pH_tuples = [pH_dict[x] for x in df["pH"].values]

temperature_map = matplotlib.cm.get_cmap('coolwarm')
colors = [temperature_map(x) for x in np.linspace(0,1,5)]
color_dict = dict(zip([25, 30, 35, 40 ,45], colors))
color_tuples = [color_dict[x] for x in df["temperature"]]

cools = ["b", "orange", "b", "orange","b", "orange","b", "orange","b", "orange","b", "orange"]

ax = plt.gca()
for i, (kcatkm, kcatkm_error, temp, color, label) in enumerate(zip(df["kcat [1/s]"], df["kcat stderr"], df["pH"] , cools, df["calibration model"].values)):

    ax.scatter(temp, kcatkm, color = color, label = label if i in [0,1] else "")
    ax.errorbar(temp, kcatkm, yerr=kcatkm_error, fmt=".", color= color, markersize=0)
plt.legend(title = "calibration model")
plt.ylabel("kcat/Km [1/s * uM]")
plt.xlabel("pH")
plt.title("Datasets of various pHs and 45°C calibrated with different standard curve fits")

In [ ]:
enzmldoc = concentration_enzymemldocs[4]

kinetics = ParameterEstimator.from_EnzymeML(enzmldoc, "s0", "substrate")
kinetics.fit_models(only_irrev_MM=True, enzyme_inactivation=False)
kinetics.visualize()